In [3]:
import sklearn
from sklearn.datasets import load_files
moviedir = r'C:/Users/Raghvendra/Desktop/movie_reviews'
# loading all files as training data. 
movie_train = load_files(moviedir, shuffle=True)
len(movie_train.data)


2000

In [4]:
# target names ("classes") are automatically generated from subfolder names
movie_train.target_names


['neg', 'pos']

In [5]:
# First file seems to be about a Schwarzenegger movie. 
movie_train.data[0][:500]


b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so cal"

In [6]:
# first file is in "neg" folder
movie_train.filenames[0]


'C:/Users/Raghvendra/Desktop/movie_reviews\\neg\\cv405_21868.txt'

In [7]:
# first file is a negative review and is mapped to 0 index 'neg' in target_names
movie_train.target[0]



0

In [8]:
#A detour: try out CountVectorizer & TF-IDF

# import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Turn off pretty printing of jupyter notebook... it generates long lines
%pprint
import nltk
nltk.download('punkt')
sents = ['A rose is a rose is a rose is a rose.',
         'Oh, what a fine day it is.',
        "It ain't over till it's over, I tell you!!"]
# Initialize a CoutVectorizer to use NLTK's tokenizer instead of its 
# default one (which ignores punctuation and stopwords). 
# Minimum document frequency set to 1. 
foovec = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize)

# sents turned into sparse vector of word frequency counts
sents_counts = foovec.fit_transform(sents)
# foovec now contains vocab dictionary which maps unique words to indexes
foovec.vocabulary_



Pretty printing has been turned OFF


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raghvendra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'a': 4, 'rose': 14, 'is': 9, '.': 3, 'oh': 12, ',': 2, 'what': 17, 'fine': 7, 'day': 6, 'it': 10, 'ai': 5, "n't": 11, 'over': 13, 'till': 16, "'s": 1, 'i': 8, 'tell': 15, 'you': 18, '!': 0}

In [9]:
# sents_counts has a dimension of 3 (document count) by 19 (# of unique words)
sents_counts.shape



(3, 19)

In [10]:
# this vector is small enough to view in full! 
sents_counts.toarray()



array([[0, 0, 0, 1, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0],
       [2, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 2, 0, 1, 1, 0, 1]],
      dtype=int64)

In [11]:
# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
sents_tfidf = tfidf_transformer.fit_transform(sents_counts)

# TF-IDF values
# raw counts have been normalized against document length, 
# terms that are found across many docs are weighted down
sents_tfidf.toarray()



array([[0.        , 0.        , 0.        , 0.13650997, 0.54603988,
        0.        , 0.        , 0.        , 0.        , 0.40952991,
        0.        , 0.        , 0.        , 0.        , 0.71797683,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.28969526, 0.28969526, 0.28969526,
        0.        , 0.38091445, 0.38091445, 0.        , 0.28969526,
        0.28969526, 0.        , 0.38091445, 0.        , 0.        ,
        0.        , 0.        , 0.38091445, 0.        ],
       [0.47282517, 0.23641258, 0.17979786, 0.        , 0.        ,
        0.23641258, 0.        , 0.        , 0.23641258, 0.        ,
        0.35959573, 0.23641258, 0.        , 0.47282517, 0.        ,
        0.23641258, 0.23641258, 0.        , 0.23641258]])

In [12]:
# initialize movie_vector object, and then turn movie train data into a vector 
movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. 82.2% acc.
# movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features = 3000) # use top 3000 words only. 78.5% acc.
movie_counts = movie_vec.fit_transform(movie_train.data)

# 'screen' is found in the corpus, mapped to index 19637
movie_vec.vocabulary_.get('screen')



19603

In [13]:
# Likewise, Mr. Steven Seagal is present...
movie_vec.vocabulary_.get('seagal')



19656

In [14]:
# huge dimensions! 2,000 documents, 25K unique terms. 
movie_counts.shape




(2000, 25279)

In [15]:
# Convert raw frequency counts into TF-IDF values
tfidf_transformer = TfidfTransformer()
movie_tfidf = tfidf_transformer.fit_transform(movie_counts)


# Same dimensions, now with tf-idf values instead of raw frequency counts
movie_tfidf.shape



(2000, 25279)

In [21]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

#Raghvendra
from sklearn.svm import LinearSVC

# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(
    movie_tfidf, movie_train.target, test_size = 0.20, random_state = 12)


# Train a Multimoda Naive Bayes classifier
clf = MultinomialNB().fit(docs_train, y_train)

# Predicting the Test set results, find accuracy
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

#Raghvendra
#clf_1 = LinearSVC().fit(docs_train, y_train)
#y1_pred = clf_1.predict(docs_test)
#sklearn.metrics.accuracy_score(y_test, y1_pred)



0.82

In [17]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[175,  31],
       [ 41, 153]], dtype=int64)

Trying classifier on fake movie reviews

In [18]:
reviews_new = ['This movie was excellent', 'Absolute joy ride', 
            'Steven Seagal was terrible', 'Steven Seagal shined through.', 
              'This was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through', 
              "We can't wait for the sequel!!", '!', '?', 'I cannot recommend this highly enough', 
              'instant classic.', 'Steven Seagal was amazing. His performance was Oscar-worthy.']
reviews_new_counts = movie_vec.transform(reviews_new)
reviews_new_tfidf = tfidf_transformer.transform(reviews_new_counts)

# have classifier make a prediction
pred = clf.predict(reviews_new_tfidf)

for review, category in zip(reviews_new, pred):
    print('%r => %s' % (review, movie_train.target_names[category]))

'This movie was excellent' => pos
'Absolute joy ride' => pos
'Steven Seagal was terrible' => neg
'Steven Seagal shined through.' => neg
'This was certainly a movie' => neg
'Two thumbs up' => neg
'I fell asleep halfway through' => neg
"We can't wait for the sequel!!" => neg
'!' => neg
'?' => neg
'I cannot recommend this highly enough' => pos
'instant classic.' => pos
'Steven Seagal was amazing. His performance was Oscar-worthy.' => neg
